In [4]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [5]:
reservas = pd.read_csv("/home/eneko/Documentos/RETO_06/datos/reservas.csv")
meteo = pd.read_csv("/home/eneko/Documentos/RETO_06/datos/meteo_limpio.csv")

In [ ]:
import pandas as pd

#Convertir las columnas de fechas a tipo fecha
reservas["checkin_time"] = pd.to_datetime(reservas["checkin_time"]).dt.date
reservas["checkout_time"] = pd.to_datetime(reservas["checkout_time"]).dt.date
meteo["fecha"] = pd.to_datetime(meteo["fecha"]).dt.date

#Asegurarse de que las columnas meteorológicas sean numéricas
meteo['tmed'] = pd.to_numeric(meteo['tmed'], errors='coerce')
meteo['prec'] = pd.to_numeric(meteo['prec'], errors='coerce')
meteo['tmin'] = pd.to_numeric(meteo['tmin'], errors='coerce')
meteo['tmax'] = pd.to_numeric(meteo['tmax'], errors='coerce')
meteo['dir'] = pd.to_numeric(meteo['dir'], errors='coerce')
meteo['velmedia'] = pd.to_numeric(meteo['velmedia'], errors='coerce')
meteo['racha'] = pd.to_numeric(meteo['racha'], errors='coerce')
meteo['sol'] = pd.to_numeric(meteo['sol'], errors='coerce')
meteo['presMax'] = pd.to_numeric(meteo['presMax'], errors='coerce')
meteo['presMin'] = pd.to_numeric(meteo['presMin'], errors='coerce')
meteo['hrMedia'] = pd.to_numeric(meteo['hrMedia'], errors='coerce')
meteo['hrMax'] = pd.to_numeric(meteo['hrMax'], errors='coerce')
meteo['hrMin'] = pd.to_numeric(meteo['hrMin'], errors='coerce')

#Definir el diccionario de agregaciones
agg_meteo = {
    "altitud": "first",  
    "tmed": "mean",
    "prec": "mean",
    "tmin": "min",
    "tmax": "max",
    "dir": "mean", 
    "velmedia": "mean",
    "racha": "max",
    "sol": "mean",
    "presMax": "max",
    "presMin": "min",
    "hrMedia": "mean",
    "hrMax": "max",
    "hrMin": "min"
}

#Resetear el índice de reservas
reservas = reservas.reset_index(drop=True)
reservas["reserva_id"] = reservas.index

#Calcular días de estancia
dias_estancia = (
    pd.to_datetime(reservas["checkout_time"]) - 
    pd.to_datetime(reservas["checkin_time"])
).dt.days + 1

#Asegurarse de que los días de estancia no sean menores a 1
dias_estancia = dias_estancia.clip(lower=1)

#Expansión de las reservas para cada día de estancia
reservas_expandidas = reservas.loc[
    reservas.index.repeat(dias_estancia)
].copy()

#Asignar las fechas a las reservas expandidas
reservas_expandidas["fecha"] = reservas_expandidas.groupby("reserva_id").cumcount()
reservas_expandidas["fecha"] = (
    pd.to_datetime(reservas_expandidas["checkin_time"]) +
    pd.to_timedelta(reservas_expandidas["fecha"], unit="D")
).dt.date

#Merge de las reservas expandidas con los datos meteorológicos
df = reservas_expandidas.merge(
    meteo,
    on=["city", "fecha"],
    how="left"
)

#Aplicar las agregaciones por reserva
meteo_por_reserva = (
    df
    .groupby("reserva_id", as_index=False)
    .agg(agg_meteo)
)

#Merge final con los datos meteorológicos agregados
reservas_final = reservas.merge(
    meteo_por_reserva,
    on="reserva_id",
    how="left"
)

#Eliminar la columna reserva_id ya que no es necesaria
reservas_final.drop(columns=["reserva_id"], inplace=True)

#Guardar el archivo CSV final
ruta_salida = "/home/eneko/Documentos/RETO_06/datos/reservas_final.csv"
reservas_final.to_csv(ruta_salida, index=False, encoding="utf-8")

print(f"Archivo generado correctamente en: {ruta_salida}")
print(f"Reservas originales: {len(reservas)}")
print(f"Reservas finales: {len(reservas_final)}")


Archivo generado correctamente en: /home/eneko/Documentos/RETO_06/datos/reservas_final.csv
Reservas originales: 52099
Reservas finales: 52099


In [8]:
reservas_final

,booked_at,checkin_time,checkout_time,lead_time,lenght_of_stay,checkin_month,checkin_day,adult_count,child_count,origin,...,tmax,dir,velmedia,racha,sol,presMax,presMin,hrMedia,hrMax,hrMin
0,2022-11-26,2023-01-01,2023-01-02,36,1,January,Sunday,1,0,channel_manager,...,24.6,24.000000,4.700000,18.9,3.150000,1029.2,1013.4,53.500000,92.0,19.0
1,2022-12-21,2023-01-01,2023-01-10,11,9,January,Sunday,1,0,channel_manager,...,24.6,18.500000,3.720000,18.9,3.790000,1034.6,1005.4,70.700000,97.0,19.0
2,2022-09-21,2023-01-01,2023-01-07,102,6,January,Sunday,2,4,channel_manager,...,24.6,17.428571,3.528571,18.9,4.357143,1034.6,1012.0,66.571429,96.0,19.0
3,2022-09-24,2023-01-01,2023-01-02,99,1,January,Sunday,2,2,channel_manager,...,24.6,24.000000,4.700000,18.9,3.150000,1029.2,1013.4,53.500000,92.0,19.0
4,2022-10-18,2023-01-01,2023-01-02,75,1,January,Sunday,4,0,channel_manager,...,24.6,24.000000,4.700000,18.9,3.150000,1029.2,1013.4,53.500000,92.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52094,2023-10-10,2023-12-31,2024-01-01,82,1,December,Sunday,4,0,channel_manager,...,12.2,26.500000,1.550000,8.3,4.250000,967.0,959.9,76.000000,96.0,51.0
52095,2023-10-13,2023-12-31,2024-01-01,79,1,December,Sunday,4,0,telephone,...,12.2,26.500000,1.550000,8.3,4.250000,967.0,959.9,76.000000,96.0,51.0
52096,2023-10-17,2023-12-31,2024-01-01,75,1,December,Sunday,5,0,direct_channel,...,12.2,26.500000,1.550000,8.3,4.250000,967.0,959.9,76.000000,96.0,51.0
52097,2023-10-17,2023-12-31,2024-01-01,75,1,December,Sunday,4,0,direct_channel,...,12.2,26.500000,1.550000,8.3,4.250000,967.0,959.9,76.000000,96.0,51.0
